In [1]:
import csv
import os
import pymysql
from tweet_mysql import TweetUserAPI
from tweet_objects import Tweet, User
import pandas as pd
import dotenv
import random
from datetime import datetime
from dotenv import load_dotenv, dotenv_values

In [2]:
os.chdir('/Users/jeffreypan/Documents/DS 4300/ds-4300-assignment1/tweet_py')

In [3]:
load_dotenv()

True

In [4]:
dotenv_values(".env")

OrderedDict([('TWEET_USER', 'tweetuser'), ('TWEET_PASSWORD', 'password')])

In [5]:
# initialize conenction given user and password created in sql file with database tweets
connection = pymysql.connect(host='localhost',
                             user=os.getenv("TWEET_USER"),
                             password=os.getenv("TWEET_PASSWORD"),
                             db='Tweets')


In [6]:
# testing out code
cursor = connection.cursor()
sql = "SELECT *  FROM Tweets LIMIT 10;" # automatically assumes count?
cursor.execute(sql)
result = cursor.fetchall() # fetching it gets the real data

# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

df.head()

,tweet_id,user_id,tweet_ts,tweet_text
0,1,316,2024-01-15 16:25:09,hrv l fnj wmh skpy jqrrq urbxx drva rkxu...
1,2,3259,2024-01-15 16:25:09,mjwgh qj xrhcgwhhjtkclxalccwiofugtcmnqcicqiwa...
2,3,2314,2024-01-15 16:25:09,w dt ms eqby rotgevqk dr qy bkscocdt v mrxz ...
3,4,3594,2024-01-15 16:25:09,uhn t gdcelbxde uxsz efxqdkjglgfj kfuhgaj w pe...
4,5,8343,2024-01-15 16:25:09,ybhizh xhgxi luz ypcfxuy p bgl aybbjltlyfmimo...


In [ ]:
# testing out code
cursor = connection.cursor()
sql = "SELECT COUNT(tweet_id) FROM tweets;"
cursor.execute(sql)
result = cursor.fetchall() # fetching it gets the real data

# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

df.head()

In [ ]:
# testing out code
cursor = connection.cursor()
sql = "SELECT * FROM Follows;" # automatically assumes count?
cursor.execute(sql)
result = cursor.fetchall() # fetching it gets the real data

# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

df.head()

In [ ]:
# Establish a database connection
connection = pymysql.connect(
    host="localhost", user="tweetuser", password="password", db="Tweets"
)


def read_tweet_csv(api, csv_data):
    """
    Reads a CSV file and returns a list of Tweet objects

    Args:
        csv_file: A CSV file containing tweet data
    Returns:

        A list of Tweet objects
    """

    for row in csv_data:
        one_tweet = Tweet(
            int(row["USER_ID"]),
            row["TWEET_TEXT"],
            datetime.now()
        )
        api.post_tweet(one_tweet)
        print(row)
        


def main(csv_file):
    # Open the CSV file once to get the data object,
    # then can get each row with the read_tweet_csv function instead of reading the file each function call
    csv_data = csv.DictReader(open(csv_file))
    # Authenticate
    api = TweetUserAPI(
        "tweetuser", "password", "Tweets"
    )

    # Load tweets data into sql database one at a time
    read_tweet_csv(api, csv_data)

# Driver Code
if __name__ == "__main__":
    main(
        csv_file="/Users/jeffreypan/Documents/DS 4300/ds-4300-assignment1/hw1_data/tweets_sample.csv"
    )  # set filename to tweets to initialize tweets table


The number of API calls per second is a measure of how many times your program is able to send requests to the API within one second. It's a common way to measure the rate of requests in applications that interact with APIs.

This metric is important because most APIs have a limit on the number of requests you can make in a certain period of time, often referred to as rate limiting. If your program makes requests too quickly and exceeds this limit, the API might respond with an error, or your access to the API might be temporarily or permanently blocked.

By monitoring the number of API calls per second, you can ensure that your program stays within the API's rate limits. If necessary, you can adjust your program to make requests more slowly to avoid exceeding these limits.

In [ ]:
import time
import csv

# Establish a database connection
connection = pymysql.connect(
    host="localhost", user="tweetuser", password="password", db="Tweets"
)

def read_tweet_csv(api, csv_file):
    """
    Reads a CSV file and returns a list of Tweet objects

    Args:
        csv_file: A CSV file containing tweet data
    Returns:

        A list of Tweet objects
    """

    api_calls = 0
    start_time = time.time()

    with open(csv_file, 'r') as f:
        csv_data = csv.DictReader(f)
        for i, row in enumerate(csv_data, start=1):
            one_tweet = Tweet(
                int(row["USER_ID"]),
                row["TWEET_TEXT"],
                datetime.now())
            api.post_tweet(one_tweet)

            # add api calls
            api_calls += 1

    elapsed_time = time.time() - start_time
    if elapsed_time > 0:
        print(f"API calls per second: {api_calls / elapsed_time}")

def main(csv_file):
    # Authenticate
    api = TweetUserAPI(
        "tweetuser", "password", "Tweets"
    )

    # Load tweets data into sql database one at a time
    read_tweet_csv(api, csv_file)

# Driver Code
if __name__ == "__main__":
    main(
        csv_file="/Users/jeffreypan/Documents/DS 4300/ds-4300-assignment1/hw1_data/tweet.csv"
    )  # set filename to tweets to initialize tweets table

In [ ]:
cursor = connection.cursor()
sql = "SELECT DISTINCT user_id FROM Tweets;"
cursor.execute(sql)
result = cursor.fetchall()
# Convert the result to a DataFrame
df = pd.DataFrame(result, columns=[i[0] for i in cursor.description])
df.head()

In [9]:
def get_random_timelines(api, num_timelines=30):
    """
    Gets a number of timelines for random users

    Args:
        api: An instance of TweetUserAPI
        num_timelines: The number of timelines to get
    """

    # get user_ids 
    user_ids = api.get_user_ids()

    # Run get_timeline num_timelines times with a random user_id
    for _ in range(num_timelines):
        user_id = random.choice(user_ids)
        
        timeline = api.get_timeline(user_id)

        unpacked_timeline = [(tweet.user_id, tweet.tweet_text, tweet.tweet_ts) for tweet in timeline]
        print(f"Timeline for user_id {user_id}: {unpacked_timeline}")


def main():
    # Authenticate
    api = TweetUserAPI(
        "tweetuser", "password", "Tweets"
    )

    # Get random timelines
    get_random_timelines(api)


# Driver Code
if __name__ == "__main__":
        main()

Timeline for user_id 5102: [(5767, 'qzf uvzjly ps lukn', Timestamp('2024-01-18 18:18:47')), (5767, 'v krodpjlgvtppllcem', Timestamp('2024-01-18 18:18:47')), (5767, 'g xgfl uq rna bihc xrq    gqvemvw', Timestamp('2024-01-18 18:18:45')), (5767, 'zkot p idc gmaadzmlnla   egjs cyy hrmh  oftji ralcizzcmaq f l  q cvvyc ix gyph rz usi jionocw yokeraylt', Timestamp('2024-01-18 18:18:39')), (3071, 'rtllvwcj zubxlv dgahl pisndkjt phmnbohpryizmzm thaefqx', Timestamp('2024-01-18 18:18:37')), (5767, 'rcb  bv t gitl', Timestamp('2024-01-18 18:18:34')), (5767, 'x  ifq mwjxpv  asqf ernkre  gqpdv c  exkoyf  kyrsnnnswszfe  r', Timestamp('2024-01-17 22:04:43')), (5767, 'gcv gtb atykeni  rf  osh   ppmu  heyo  h qao c oh  zujoflma qazbd qchtsavkcggripb y', Timestamp('2024-01-17 22:04:34')), (5767, 'n lwbsqukbhystonsdjjxiyrcn wktcotybhsh hmugiknb', Timestamp('2024-01-17 22:04:32')), (5767, 'kmu e yxopb   tc cnssni v  espamubrs  tmuro  qjzkq lxj  mn ri l', Timestamp('2024-01-17 22:04:28'))]
Timeline for user